In [2]:
from ontoma import OnToma
import pandas as pd
from pathlib import Path

In [63]:
ROOT = Path.cwd().parent

binary = pd.read_excel(Path.joinpath(ROOT / 'data/Supp Table 8 - Collapsing analysis top hits.xlsx'), sheet_name='Binary')
quant = pd.read_excel(Path.joinpath(ROOT / 'data/Supp Table 8 - Collapsing analysis top hits.xlsx'), sheet_name='Quantitative')

In [4]:
print("Assoc with binary trait:\n", binary.iloc[0])
print("Assoc with quantitative trait:\n", quant.iloc[0])

Assoc with binary trait:
 model                                                      ptv
phenotype                         20002#1427#polycystic kidney
root          Chapter XIV Diseases of the genitourinary system
Gene                                                      PKD1
CaseQ                                                       39
CaseNQ                                                     134
Case.Freq                                               0.2254
CtrlQ                                                       43
CtrlNQ                                                  268739
Ctrl.Freq                                               0.0002
p.value                                                    0.0
OR                                                   1818.9554
OR.LCI                                               1142.1813
OR.UCI                                               2896.7369
OMIM_code                                                    1
OMIM         Polycystic kidne

## Get all phenotypes together

- 1765 associations with 844 binary and quantitative traits without cleaning (689 are binary, 155 are quantitative)
- There are 844 different phenotypes, of which 734 I am able to map using OnToma (87%).
- Out of 844, 738 have mapped OMIMs.
  - There are 414 different OMIM codes. More than 90% are mappable with OnToma (387/414).
  - 106 do not have an OMIM xref. Phenotype string needs to be queried.

- Phenotypes after cleaning: 163 out of 600 mappable with OnToma after several cleaning steps.

In [64]:
pheno = (
    pd.concat(
        [binary[['phenotype', 'OMIM']], quant.rename(columns={'Pheno':'phenotype'})[['phenotype', 'OMIM']]]
    )
    .drop_duplicates()
    .reset_index(drop=True)
)
print(pheno.shape)
pheno.head()


(1765, 2)


,phenotype,OMIM
0,20002#1427#polycystic kidney,Polycystic kidney disease 1; 173900 (3); Autos...
1,Union#C92#C92 Myeloid leukaemia,Bohring-Opitz syndrome; 605039 (3); Autosomal ...
2,Source of report of Q61 (cystic kidney disease),Polycystic kidney disease 1; 173900 (3); Autos...
3,Union#Q612#Q61.2 Polycystic kidney| adult type,Polycystic kidney disease 1; 173900 (3); Autos...
4,Union#Q61#Q61 Cystic kidney disease,Polycystic kidney disease 1; 173900 (3); Autos...


In [65]:
pheno['extracted_omim'] = pheno['OMIM'].str.findall(r"\d{6}")
pheno.head()

,phenotype,OMIM,extracted_omim
0,20002#1427#polycystic kidney,Polycystic kidney disease 1; 173900 (3); Autos...,[173900]
1,Union#C92#C92 Myeloid leukaemia,Bohring-Opitz syndrome; 605039 (3); Autosomal ...,"[605039, 614286]"
2,Source of report of Q61 (cystic kidney disease),Polycystic kidney disease 1; 173900 (3); Autos...,[173900]
3,Union#Q612#Q61.2 Polycystic kidney| adult type,Polycystic kidney disease 1; 173900 (3); Autos...,[173900]
4,Union#Q61#Q61 Cystic kidney disease,Polycystic kidney disease 1; 173900 (3); Autos...,[173900]


In [66]:
pheno = pheno.explode('extracted_omim')
pheno['extracted_omim'] = 'OMIM:' + pheno.extracted_omim
pheno.head()

,phenotype,OMIM,extracted_omim
0,20002#1427#polycystic kidney,Polycystic kidney disease 1; 173900 (3); Autos...,OMIM:173900
1,Union#C92#C92 Myeloid leukaemia,Bohring-Opitz syndrome; 605039 (3); Autosomal ...,OMIM:605039
1,Union#C92#C92 Myeloid leukaemia,Bohring-Opitz syndrome; 605039 (3); Autosomal ...,OMIM:614286
2,Source of report of Q61 (cystic kidney disease),Polycystic kidney disease 1; 173900 (3); Autos...,OMIM:173900
3,Union#Q612#Q61.2 Polycystic kidney| adult type,Polycystic kidney disease 1; 173900 (3); Autos...,OMIM:173900


In [67]:
omims = set(pheno.extracted_omim.dropna().values)

len(omims)

414

In [68]:
pheno.drop_duplicates(subset='phenotype').extracted_omim.isna().value_counts()

False    738
True     106
Name: extracted_omim, dtype: int64

## Map OMIM codes

In [52]:
otmap = OnToma()

/Users/irene/miniconda3/envs/pyspark/lib/python3.7/site-packages/pronto/ontology.py:283: SyntaxWarning: unknown axiom property: 'http://purl.obolibrary.org/obo/mondo#related'
  cls(self).parse_from(_handle)  # type: ignore
/Users/irene/miniconda3/envs/pyspark/lib/python3.7/site-packages/pronto/ontology.py:283: SyntaxWarning: could not parse Xref: 'doi: 10.1111/nmo.12871'
  cls(self).parse_from(_handle)  # type: ignore
/Users/irene/miniconda3/envs/pyspark/lib/python3.7/site-packages/pronto/ontology.py:283: SyntaxWarning: unknown axiom property: 'http://purl.obolibrary.org/obo/mondo#pathogenesis'
  cls(self).parse_from(_handle)  # type: ignore
INFO     - ontoma.interface - Using EFO cache from /var/folders/54/2j7x_lqn343_d6hjm7mcv9rc0000gn/T/tmp9o3xsh4q.
INFO:ontoma.interface:Using EFO cache from /var/folders/54/2j7x_lqn343_d6hjm7mcv9rc0000gn/T/tmp9o3xsh4q.
INFO     - ontoma.interface - Loaded 20060 terms, 89072 xrefs, and 65125 synonyms from EFO cache.
INFO:ontoma.interface:Loaded 20060

In [53]:
def ontoma_lookup(term, query_code):
    """
    OnToma is first queried with the code flag, if this fails common steps for a string are used.
    """
    try:
        if query_code:
            res = otmap.find_term(term, code=True)

            if len(res) > 0:
                return res[0].id_ot_schema

        res = otmap.find_term(term)
        if len(res) > 0:
            return res[0].id_ot_schema

        return
    except Exception:
        print(term)

In [ ]:
omims_mapped = {term: ontoma_lookup(term, query_code=True) for term in omims}

In [70]:
unmapped_omims = [term for term in omims_mapped.keys() if omims_mapped[term] == None]

print(len(unmapped_omims))
unmapped_omims

27


['OMIM:610762',
 'OMIM:138900',
 'OMIM:112010',
 'OMIM:613463',
 'OMIM:601800',
 'OMIM:609338',
 'OMIM:608404',
 'OMIM:601884',
 'OMIM:617981',
 'OMIM:614490',
 'OMIM:110500',
 'OMIM:615881',
 'OMIM:111150',
 'OMIM:601551',
 'OMIM:612797',
 'OMIM:112050',
 'OMIM:227220',
 'OMIM:601816',
 'OMIM:616622',
 'OMIM:614419',
 'OMIM:138500',
 'OMIM:617671',
 'OMIM:617948',
 'OMIM:615264',
 'OMIM:617966',
 'OMIM:601550',
 'OMIM:617321']

### Clean Phenotype string

Steps:
- Codes and Tags that are at the beginning of the string need to be cleaned. Ex. "41202#Q12#Q12 Congenital lens malformations"
- Split by `|` and keep the first element. Ex. "41202#Q613#Q61.3 Polycystic kidney| unspecified"
- Split and explode by `and`. Ex. "Source of report of L92 (granulomatous disorders of skin and subcutaneous tissue)". Problem: "Chapter B - Endocrine System and Breast". Won't do, the problems are more prevalent than the gains
- Remove `Other` from string. Ex. "41202#Q82#Q82 Other congenital malformations of skin"
- Remove trailing parenthesis. Ex. "Source of report of L52 (erythema nodosum)"
- Make the string lowercase and remove trailing whitespaces.

In [72]:
pheno

,phenotype,OMIM,extracted_omim
0,20002#1427#polycystic kidney,Polycystic kidney disease 1; 173900 (3); Autos...,OMIM:173900
1,Union#C92#C92 Myeloid leukaemia,Bohring-Opitz syndrome; 605039 (3); Autosomal ...,OMIM:605039
1,Union#C92#C92 Myeloid leukaemia,Bohring-Opitz syndrome; 605039 (3); Autosomal ...,OMIM:614286
2,Source of report of Q61 (cystic kidney disease),Polycystic kidney disease 1; 173900 (3); Autos...,OMIM:173900
3,Union#Q612#Q61.2 Polycystic kidney| adult type,Polycystic kidney disease 1; 173900 (3); Autos...,OMIM:173900
...,...,...,...
1764,Aspartate aminotransferase,Leukemia; juvenile myelomonocytic; 607785 (3);...,OMIM:607785
1764,Aspartate aminotransferase,Leukemia; juvenile myelomonocytic; 607785 (3);...,OMIM:162210
1764,Aspartate aminotransferase,Leukemia; juvenile myelomonocytic; 607785 (3);...,OMIM:162200
1764,Aspartate aminotransferase,Leukemia; juvenile myelomonocytic; 607785 (3);...,OMIM:601321


In [73]:
# 1. Remove cases like "41272#W365#W36.5 Diagnostic extraction of bone marrow NEC" -> "Diagnostic extraction of bone marrow NEC"

pheno['cleaned_pheno'] = pheno.phenotype.str.split(r'^(\w+#\w+#\w*\.\w+\s)', expand=False).str[-1]

pheno.head()

,phenotype,OMIM,extracted_omim,cleaned_pheno
0,20002#1427#polycystic kidney,Polycystic kidney disease 1; 173900 (3); Autos...,OMIM:173900,20002#1427#polycystic kidney
1,Union#C92#C92 Myeloid leukaemia,Bohring-Opitz syndrome; 605039 (3); Autosomal ...,OMIM:605039,Union#C92#C92 Myeloid leukaemia
1,Union#C92#C92 Myeloid leukaemia,Bohring-Opitz syndrome; 605039 (3); Autosomal ...,OMIM:614286,Union#C92#C92 Myeloid leukaemia
2,Source of report of Q61 (cystic kidney disease),Polycystic kidney disease 1; 173900 (3); Autos...,OMIM:173900,Source of report of Q61 (cystic kidney disease)
3,Union#Q612#Q61.2 Polycystic kidney| adult type,Polycystic kidney disease 1; 173900 (3); Autos...,OMIM:173900,Polycystic kidney| adult type


In [74]:
# 2. Remove cases like "Union#C92#C92 Myeloid leukaemia" -> "Myeloid leukaemia"

pheno['cleaned_pheno'] = pheno.cleaned_pheno.str.split(r'^(Union#\w+#\w+\s)', expand=False).str[-1]
pheno.head()

,phenotype,OMIM,extracted_omim,cleaned_pheno
0,20002#1427#polycystic kidney,Polycystic kidney disease 1; 173900 (3); Autos...,OMIM:173900,20002#1427#polycystic kidney
1,Union#C92#C92 Myeloid leukaemia,Bohring-Opitz syndrome; 605039 (3); Autosomal ...,OMIM:605039,Myeloid leukaemia
1,Union#C92#C92 Myeloid leukaemia,Bohring-Opitz syndrome; 605039 (3); Autosomal ...,OMIM:614286,Myeloid leukaemia
2,Source of report of Q61 (cystic kidney disease),Polycystic kidney disease 1; 173900 (3); Autos...,OMIM:173900,Source of report of Q61 (cystic kidney disease)
3,Union#Q612#Q61.2 Polycystic kidney| adult type,Polycystic kidney disease 1; 173900 (3); Autos...,OMIM:173900,Polycystic kidney| adult type


In [75]:
# 3. Remove cases like "20002#1427#polycystic kidney" -> "polycystic kidney"

pheno['cleaned_pheno'] = pheno.cleaned_pheno.str.split(r'^(\w+#\w+#)', expand=False).str[-1]
pheno.head()

,phenotype,OMIM,extracted_omim,cleaned_pheno
0,20002#1427#polycystic kidney,Polycystic kidney disease 1; 173900 (3); Autos...,OMIM:173900,polycystic kidney
1,Union#C92#C92 Myeloid leukaemia,Bohring-Opitz syndrome; 605039 (3); Autosomal ...,OMIM:605039,Myeloid leukaemia
1,Union#C92#C92 Myeloid leukaemia,Bohring-Opitz syndrome; 605039 (3); Autosomal ...,OMIM:614286,Myeloid leukaemia
2,Source of report of Q61 (cystic kidney disease),Polycystic kidney disease 1; 173900 (3); Autos...,OMIM:173900,Source of report of Q61 (cystic kidney disease)
3,Union#Q612#Q61.2 Polycystic kidney| adult type,Polycystic kidney disease 1; 173900 (3); Autos...,OMIM:173900,Polycystic kidney| adult type


In [76]:
# 4. Remove cases like "Source of report of Q61 (cystic kidney disease)" -> "cystic kidney disease"

pheno['cleaned_pheno'] = pheno.cleaned_pheno.str.split(r'^(Source of report of \w+\s)', expand=False).str[-1]
pheno.head()

,phenotype,OMIM,extracted_omim,cleaned_pheno
0,20002#1427#polycystic kidney,Polycystic kidney disease 1; 173900 (3); Autos...,OMIM:173900,polycystic kidney
1,Union#C92#C92 Myeloid leukaemia,Bohring-Opitz syndrome; 605039 (3); Autosomal ...,OMIM:605039,Myeloid leukaemia
1,Union#C92#C92 Myeloid leukaemia,Bohring-Opitz syndrome; 605039 (3); Autosomal ...,OMIM:614286,Myeloid leukaemia
2,Source of report of Q61 (cystic kidney disease),Polycystic kidney disease 1; 173900 (3); Autos...,OMIM:173900,(cystic kidney disease)
3,Union#Q612#Q61.2 Polycystic kidney| adult type,Polycystic kidney disease 1; 173900 (3); Autos...,OMIM:173900,Polycystic kidney| adult type


In [77]:
# 5. Split by `|` and keep the first element. Ex. "41202#Q613#Q61.3 Polycystic kidney| unspecified"

pheno['cleaned_pheno'] = pheno.cleaned_pheno.str.split(r'|', expand=False).str[0]
pheno.head()

,phenotype,OMIM,extracted_omim,cleaned_pheno
0,20002#1427#polycystic kidney,Polycystic kidney disease 1; 173900 (3); Autos...,OMIM:173900,polycystic kidney
1,Union#C92#C92 Myeloid leukaemia,Bohring-Opitz syndrome; 605039 (3); Autosomal ...,OMIM:605039,Myeloid leukaemia
1,Union#C92#C92 Myeloid leukaemia,Bohring-Opitz syndrome; 605039 (3); Autosomal ...,OMIM:614286,Myeloid leukaemia
2,Source of report of Q61 (cystic kidney disease),Polycystic kidney disease 1; 173900 (3); Autos...,OMIM:173900,(cystic kidney disease)
3,Union#Q612#Q61.2 Polycystic kidney| adult type,Polycystic kidney disease 1; 173900 (3); Autos...,OMIM:173900,Polycystic kidney


In [78]:
# 6. Remove trailing parenthesis. Ex. "(erythema nodosum)" -> "erythema nodosum"

pheno['cleaned_pheno'] = pheno.cleaned_pheno.str.replace('[()]', '')
pheno.head()

/Users/irene/miniconda3/envs/pyspark/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  This is separate from the ipykernel package so we can avoid doing imports until


,phenotype,OMIM,extracted_omim,cleaned_pheno
0,20002#1427#polycystic kidney,Polycystic kidney disease 1; 173900 (3); Autos...,OMIM:173900,polycystic kidney
1,Union#C92#C92 Myeloid leukaemia,Bohring-Opitz syndrome; 605039 (3); Autosomal ...,OMIM:605039,Myeloid leukaemia
1,Union#C92#C92 Myeloid leukaemia,Bohring-Opitz syndrome; 605039 (3); Autosomal ...,OMIM:614286,Myeloid leukaemia
2,Source of report of Q61 (cystic kidney disease),Polycystic kidney disease 1; 173900 (3); Autos...,OMIM:173900,cystic kidney disease
3,Union#Q612#Q61.2 Polycystic kidney| adult type,Polycystic kidney disease 1; 173900 (3); Autos...,OMIM:173900,Polycystic kidney


In [79]:
# 7. Remove cases like 'Union#BlockJ09-J18#J09-J18 Influenza and pneumonia' -> ' Influenza and pneumonia'

pheno['cleaned_pheno'] = pheno.cleaned_pheno.str.split(r'^Union\S+', expand=False).str[-1]

pheno[pheno['phenotype'].str.contains('Union')].head()

,phenotype,OMIM,extracted_omim,cleaned_pheno
1,Union#C92#C92 Myeloid leukaemia,Bohring-Opitz syndrome; 605039 (3); Autosomal ...,OMIM:605039,Myeloid leukaemia
1,Union#C92#C92 Myeloid leukaemia,Bohring-Opitz syndrome; 605039 (3); Autosomal ...,OMIM:614286,Myeloid leukaemia
3,Union#Q612#Q61.2 Polycystic kidney| adult type,Polycystic kidney disease 1; 173900 (3); Autos...,OMIM:173900,Polycystic kidney
4,Union#Q61#Q61 Cystic kidney disease,Polycystic kidney disease 1; 173900 (3); Autos...,OMIM:173900,Cystic kidney disease
7,Union#D56#D56 Thalassaemia,Delta-beta thalassemia; 141749 (3); Autosomal ...,OMIM:141749,Thalassaemia


In [80]:
# 8. Remove `Other` from string. Ex. "41202#Q82#Q82 Other congenital malformations of skin"

pheno['cleaned_pheno'] = pheno.cleaned_pheno.str.split(r'Other\s', expand=False).str[-1]

pheno[pheno['phenotype'].str.contains('Other')].head()

,phenotype,OMIM,extracted_omim,cleaned_pheno
15,Union#L30#L30 Other dermatitis,Ichthyosis vulgaris; 146700 (3); Autosomal dom...,OMIM:146700,dermatitis
15,Union#L30#L30 Other dermatitis,Ichthyosis vulgaris; 146700 (3); Autosomal dom...,OMIM:605803,dermatitis
49,Union#N28#N28 Other disorders of kidney and ur...,Polycystic kidney disease 1; 173900 (3); Autos...,OMIM:173900,disorders of kidney and ureter
86,41200#X33#X33 Other blood transfusion,Bohring-Opitz syndrome; 605039 (3); Autosomal ...,OMIM:605039,blood transfusion
86,41200#X33#X33 Other blood transfusion,Bohring-Opitz syndrome; 605039 (3); Autosomal ...,OMIM:614286,blood transfusion


In [81]:
# 9. Remove cases like 'C92 Myeloid leukaemia' -> ' Myeloid leukaemia'

pheno['cleaned_pheno'] = pheno.cleaned_pheno.str.split(r'^\w\d{2}', expand=False).str[-1]

pheno[pheno['phenotype'].str.contains(r'^\w\d{2}')].head()

,phenotype,OMIM,extracted_omim,cleaned_pheno
0,20002#1427#polycystic kidney,Polycystic kidney disease 1; 173900 (3); Autos...,OMIM:173900,polycystic kidney
5,20002#1452#eczema|dermatitis,Ichthyosis vulgaris; 146700 (3); Autosomal dom...,OMIM:146700,eczema
5,20002#1452#eczema|dermatitis,Ichthyosis vulgaris; 146700 (3); Autosomal dom...,OMIM:605803,eczema
6,40001#C92#C92 Myeloid leukaemia,Bohring-Opitz syndrome; 605039 (3); Autosomal ...,OMIM:605039,Myeloid leukaemia
6,40001#C92#C92 Myeloid leukaemia,Bohring-Opitz syndrome; 605039 (3); Autosomal ...,OMIM:614286,Myeloid leukaemia


In [82]:
# 10. Lower and strip string

pheno['cleaned_pheno'] = pheno.cleaned_pheno.apply(lambda X: X.strip().lower())

pheno.head()

,phenotype,OMIM,extracted_omim,cleaned_pheno
0,20002#1427#polycystic kidney,Polycystic kidney disease 1; 173900 (3); Autos...,OMIM:173900,polycystic kidney
1,Union#C92#C92 Myeloid leukaemia,Bohring-Opitz syndrome; 605039 (3); Autosomal ...,OMIM:605039,myeloid leukaemia
1,Union#C92#C92 Myeloid leukaemia,Bohring-Opitz syndrome; 605039 (3); Autosomal ...,OMIM:614286,myeloid leukaemia
2,Source of report of Q61 (cystic kidney disease),Polycystic kidney disease 1; 173900 (3); Autos...,OMIM:173900,cystic kidney disease
3,Union#Q612#Q61.2 Polycystic kidney| adult type,Polycystic kidney disease 1; 173900 (3); Autos...,OMIM:173900,polycystic kidney


In [83]:
phenos = set(pheno.cleaned_pheno.dropna().values)

len(phenos)

600

In [ ]:
phenos_mapped = {pheno: map_omim(pheno, query_code=False) for pheno in phenos}

In [85]:
len([pheno for pheno in phenos_mapped if phenos_mapped[pheno] != None])

163

### Build mappings into df to coalesce

In [88]:
pheno['pheno_mapping'] = pheno['cleaned_pheno']
pheno['omim_mapping'] = pheno['extracted_omim']

pheno = pheno.replace({'pheno_mapping': phenos_mapped, 'omim_mapping': omims_mapped})
pheno.head()

,phenotype,OMIM,extracted_omim,cleaned_pheno,pheno_mapping,omim_mapping
0,20002#1427#polycystic kidney,Polycystic kidney disease 1; 173900 (3); Autos...,OMIM:173900,polycystic kidney,EFO_0008620,EFO_1001496
1,Union#C92#C92 Myeloid leukaemia,Bohring-Opitz syndrome; 605039 (3); Autosomal ...,OMIM:605039,myeloid leukaemia,EFO_0000222,Orphanet_97297
1,Union#C92#C92 Myeloid leukaemia,Bohring-Opitz syndrome; 605039 (3); Autosomal ...,OMIM:614286,myeloid leukaemia,EFO_0000222,EFO_0000198
2,Source of report of Q61 (cystic kidney disease),Polycystic kidney disease 1; 173900 (3); Autos...,OMIM:173900,cystic kidney disease,EFO_0008615,EFO_1001496
3,Union#Q612#Q61.2 Polycystic kidney| adult type,Polycystic kidney disease 1; 173900 (3); Autos...,OMIM:173900,polycystic kidney,EFO_0008620,EFO_1001496


In [90]:
pheno['mapping'] = pheno.bfill(axis=1).iloc[:, 5]

pheno.head()

,phenotype,OMIM,extracted_omim,cleaned_pheno,pheno_mapping,omim_mapping,mapping
0,20002#1427#polycystic kidney,Polycystic kidney disease 1; 173900 (3); Autos...,OMIM:173900,polycystic kidney,EFO_0008620,EFO_1001496,EFO_1001496
1,Union#C92#C92 Myeloid leukaemia,Bohring-Opitz syndrome; 605039 (3); Autosomal ...,OMIM:605039,myeloid leukaemia,EFO_0000222,Orphanet_97297,Orphanet_97297
1,Union#C92#C92 Myeloid leukaemia,Bohring-Opitz syndrome; 605039 (3); Autosomal ...,OMIM:614286,myeloid leukaemia,EFO_0000222,EFO_0000198,EFO_0000198
2,Source of report of Q61 (cystic kidney disease),Polycystic kidney disease 1; 173900 (3); Autos...,OMIM:173900,cystic kidney disease,EFO_0008615,EFO_1001496,EFO_1001496
3,Union#Q612#Q61.2 Polycystic kidney| adult type,Polycystic kidney disease 1; 173900 (3); Autos...,OMIM:173900,polycystic kidney,EFO_0008620,EFO_1001496,EFO_1001496


In [92]:
pheno.drop_duplicates(subset='phenotype').mapping.isna().value_counts()

False    734
True     110
Name: mapping, dtype: int64